# Trabalho Prático - Cálculo Numérico (SME0104)

Este notebook contém a resolução completa do trabalho prático da disciplina de Cálculo Numérico (SME0104), ministrada pela professora Cynthia Lage Ferreira, conforme as orientações fornecidas.

Todas as questões foram resolvidas em Python, com ênfase na implementação dos métodos numéricos abordados em aula, análise teórica e validação dos resultados obtidos.

**Autores**:  
Luiz Felipe Diniz Costa - 13782032  
Pedro Lucas Figueiredo Bahiense - 14675458

> **Observação**  
> O Trabalho incluirá um resumo teórico para cada questão, conforme sugerido pela professora.